In [1]:
import pandas as pd
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from pandas.api.types import is_datetime64_ns_dtype

In [2]:
df_tickets = pd.read_json('tickets.json')
df_users = pd.read_json('users.json')
df_orgs = pd.read_json('organizations.json')

query_table = None
query_column = None
query_value = None

In [3]:
def get_column_type(target_df, column):
    if column in target_df.columns:
        if is_string_dtype(target_df[column]):
            return 'string'
        elif is_numeric_dtype(target_df[column]):
            return 'int'
        elif is_datetime64_ns_dtype(target_df[column]):
            return 'datetime'
        else:
            return 'unknown'
    else:
        return 'unknown'

In [4]:
def print_formated_result(result):
    if result.empty:
        print('Search {0} for {1} with a value of {2}\nNo results found'.format(query_table, query_column, query_value[1]))
    else:
        display(result)

In [5]:
def equal_query(target_df, column, column_type, value):
    if column_type == 'int':
        value = int(value)
        
    result = target_df[target_df[column] == value]
    
    print_formated_result(result)

In [6]:
def like_query(target_df, column, value):
    result = target_df[target_df[column].str.contains(value, na=False)]
    
    print_formated_result(result)

In [7]:
def parse_query(input_2, input_3, input_4):
    CONST_TABLES_DICT = {1: ['users', 'users'], 2: ['tickets', 'tickets'], 3: ['orgs', 'organizations']}
    
    target_df = globals()['df_' + CONST_TABLES_DICT[input_2][0]]
    
    global query_table
    query_table = CONST_TABLES_DICT[input_2][1]
    
    global query_column
    query_column =  input_3
    
    global query_value
    query_value = input_4.split('@@')
    
    column_type = get_column_type(target_df, query_column)
    
    if query_value[0] == 'equal':
        equal_query(target_df, query_column, column_type, query_value[1])
    elif query_value[0] == 'like':
        like_query(target_df, query_column, query_value[1])
    ## To-DO: Add more type of query functions, and also will support multiple columns query
    ## Main problems that would be resolved in the next phase is to query between two datetimes

In [8]:
parse_query(2, '_id', 'equal@@916aab4a-0577-40cf-8f56-a45912a6ac23')

,_id,url,external_id,created_at,type,subject,description,priority,status,submitter_id,assignee_id,organization_id,tags,has_incidents,due_at,via
69,916aab4a-0577-40cf-8f56-a45912a6ac23,http://initech.zendesk.com/api/v2/tickets/916a...,9fbaddb4-d72f-4270-baf9-dba1de59bb4b,2016-05-14T11:36:47 -10:00,task,A Nuisance in Togo,Magna dolor do enim velit incididunt reprehend...,low,pending,19,39.0,110.0,"[Mississippi, Marshall Islands, South Dakota, ...",False,2016-08-15 01:19:11-10:00,chat


In [9]:
parse_query(2, 'organization_id', 'equal@@110')

,_id,url,external_id,created_at,type,subject,description,priority,status,submitter_id,assignee_id,organization_id,tags,has_incidents,due_at,via
69,916aab4a-0577-40cf-8f56-a45912a6ac23,http://initech.zendesk.com/api/v2/tickets/916a...,9fbaddb4-d72f-4270-baf9-dba1de59bb4b,2016-05-14T11:36:47 -10:00,task,A Nuisance in Togo,Magna dolor do enim velit incididunt reprehend...,low,pending,19,39.0,110.0,"[Mississippi, Marshall Islands, South Dakota, ...",False,2016-08-15 01:19:11-10:00,chat
76,c496e355-4400-4baa-b8ca-bb2edd270c43,http://initech.zendesk.com/api/v2/tickets/c496...,4cc40f31-ddd0-4324-ada8-f298531833db,2016-02-26T06:45:12 -11:00,incident,A Catastrophe in Moldova,Laboris dolor aliquip adipisicing nisi amet al...,urgent,hold,3,33.0,110.0,"[Oregon, Arizona, Delaware, New Hampshire]",True,2016-08-12 06:24:57-10:00,voice
85,6ed590ac-e385-46e2-a27a-50628a658168,http://initech.zendesk.com/api/v2/tickets/6ed5...,0142fe07-2e8d-47ef-8cd8-8508810f13f7,2016-03-03T05:35:49 -11:00,task,A Problem in Uzbekistan,Anim do est ea ex minim voluptate laborum culp...,normal,open,72,44.0,110.0,"[Illinois, Fédératéd Statés Of Micronésia, Rho...",False,2016-08-22 06:11:46-10:00,web
145,ccf4c82c-f572-4fd2-82a6-11d6055929b8,http://initech.zendesk.com/api/v2/tickets/ccf4...,badc534c-9f11-45a9-93f2-509b86ed31fa,2016-01-20T12:45:55 -11:00,problem,A Catastrophe in Lebanon,Nisi elit minim excepteur qui. Non incididunt ...,high,hold,35,52.0,110.0,"[Ohio, Pennsylvania, American Samoa, Northern ...",True,2016-08-14 10:58:25-10:00,chat
150,140e0cd4-c31b-4e90-833d-c42a12d4b713,http://initech.zendesk.com/api/v2/tickets/140e...,dd0229db-7361-4d4e-bbe3-235024436c4b,2016-03-12T11:33:46 -11:00,incident,A Nuisance in Macedonia,Incididunt et laborum tempor nulla. Exercitati...,normal,pending,45,24.0,110.0,"[Alaska, Maryland, Iowa, North Dakota]",True,2016-08-10 07:48:01-10:00,voice


In [10]:
parse_query(1, 'alias', 'like@@Mr')

,_id,url,external_id,name,alias,created_at,active,verified,shared,locale,timezone,last_login_at,email,phone,signature,organization_id,tags,suspended,role
3,4,http://initech.zendesk.com/api/v2/users/4.json,37c9aef5-cf01-4b07-af24-c6c49ac1d1c7,Rose Newton,Mr Cardenas,2016-02-09T07:52:10 -11:00,True,1.0,True,de-CH,Netherlands,2012-09-25T01:32:46 -10:00,cardenasnewton@flotonic.com,8685-482-450,Don't Worry Be Happy!,122.0,"[Gallina, Glenshaw, Rowe, Babb]",True,end-user
4,5,http://initech.zendesk.com/api/v2/users/5.json,29c18801-fb42-433d-8674-f37d63e637df,Loraine Pittman,Mr Ola,2016-06-12T08:49:19 -10:00,True,0.0,False,zh-CN,Monaco,2013-07-03T06:59:27 -10:00,olapittman@flotonic.com,9805-292-618,Don't Worry Be Happy!,101.0,"[Frizzleburg, Forestburg, Sandston, Delco]",False,admin
8,9,http://initech.zendesk.com/api/v2/users/9.json,215e9cb6-cd30-444d-95f3-4c172d68859d,Josefa Mcfadden,Mr Casey,2016-03-26T04:11:05 -11:00,True,0.0,False,zh-CN,"France, Metropolitan",2013-04-25T09:04:57 -10:00,caseymcfadden@flotonic.com,8715-273-472,Don't Worry Be Happy!,123.0,"[Fresno, Shawmut, Buxton, Winchester]",True,agent
9,10,http://initech.zendesk.com/api/v2/users/10.json,f744706e-df3d-4d51-ad18-c63e41be5cc0,Kari Vinson,Mr Webb,2016-02-08T04:32:38 -11:00,True,0.0,True,en-AU,Germany,2014-12-26T04:54:30 -11:00,webbvinson@flotonic.com,8235-883-140,Don't Worry Be Happy!,105.0,"[Moraida, Beechmont, Jeff, Starks]",False,end-user
11,12,http://initech.zendesk.com/api/v2/users/12.json,38899b1e-89ca-43e7-b039-e3c88525f0d2,Watkins Hammond,Mr Sally,2016-07-21T12:26:16 -10:00,False,0.0,False,en-AU,United Kingdom,2012-12-29T07:59:56 -11:00,sallyhammond@flotonic.com,8144-293-283,Don't Worry Be Happy!,110.0,"[Bonanza, Balm, Fulford, Austinburg]",False,end-user
12,13,http://initech.zendesk.com/api/v2/users/13.json,dce36d80-f143-4b1e-8a2e-47651e367a4e,Wade Moore,Mr Vera,2016-06-02T01:11:48 -10:00,True,1.0,False,en-AU,Saudi Arabia,2012-08-03T05:09:18 -10:00,veramoore@flotonic.com,8395-652-590,Don't Worry Be Happy!,112.0,"[Bartonsville, Frystown, Sparkill, Dola]",True,end-user
18,19,http://initech.zendesk.com/api/v2/users/19.json,68e35e26-7b1f-46ec-a9e5-3edcbcf2aeb9,Francis Rodrigüez,Mr Lea,2016-05-05T01:56:24 -10:00,False,0.0,False,zh-CN,Brazil,2012-05-25T01:55:34 -10:00,NaN,8275-873-442,Don't Worry Be Happy!,124.0,"[Vicksburg, Kilbourne, Gorham, Gloucester]",False,agent
28,29,http://initech.zendesk.com/api/v2/users/29.json,5cf7c032-b3cb-4c87-afa1-57fc9f94e9a1,Herrera Norman,Mr Vance,2016-03-17T06:09:57 -11:00,False,0.0,True,en-AU,Zimbabwe,2016-05-17T03:03:05 -10:00,vancenorman@flotonic.com,9444-743-342,Don't Worry Be Happy!,101.0,"[Tilden, Layhill, Franklin, Allensworth]",False,end-user
29,30,http://initech.zendesk.com/api/v2/users/30.json,33ee4880-9ff2-4233-ae8a-24b9d5cc03b2,Debra William,Mr Murray,2016-03-05T05:38:13 -11:00,True,0.0,True,zh-CN,Zambia,2015-01-22T06:48:31 -11:00,murraywilliam@flotonic.com,9734-862-052,Don't Worry Be Happy!,116.0,"[Chesterfield, Brutus, Echo, Valmy]",False,agent
31,32,http://initech.zendesk.com/api/v2/users/32.json,b8ef8d96-4d64-49a3-b2c6-b04bdc23ba01,Lee Dotson,Mr Eve,2016-07-17T02:09:22 -10:00,True,1.0,False,zh-CN,France,2016-08-21T09:09:09 -10:00,evedotson@flotonic.com,8485-142-308,Don't Worry Be Happy!,105.0,"[Tetherow, Hailesboro, Barstow, Fruitdale]",False,agent


In [11]:
parse_query(1, 'name', 'equal@@George')

Search users for name with a value of George
No results found
